In [9]:
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from collections import Counter
from matplotlib import pyplot as plt

In [10]:
pattern = r'TEXT: (.+)'
with open('collections/sample.txt', 'r') as file:
    txt = file.read()
    documents = re.findall(pattern, txt)

# print(documents)

In [11]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from stemming.porter2 import stem 

tokenization = RegexpTokenizer(r"\w+")
new = []

with open('stopWords.txt', 'r') as file:
    stopWords = file.read().split()

def preprocess(stp, stm, documents):
    for doc in documents:
        # Tokenization
        tokenized = tokenization.tokenize(doc)
        
        # Casefolding
        lower = [word.lower() for word in tokenized]
        
        if stp:
            # Stopwords removal
            lower = [word for word in lower if not word in stopWords]
        
        if stm:
            # Stemming using the stemming library
            lower = [stem(word) for word in lower]
        
        new.append(lower)

preprocess(True, True, documents)
# print(new)
for list in new:
    print(list)


['like', 'wink', 'like', 'drink']
['like', 'drink', 'drink', 'drink']
['thing', 'like', 'drink', 'ink']
['ink', 'like', 'drink', 'pink']
['like', 'wink', 'drink', 'pink', 'ink']


In [12]:
# Concatenate tokens
final=[]
for i in new:
    final.append(' '.join(i))
print(final)  

['like wink like drink', 'like drink drink drink', 'thing like drink ink', 'ink like drink pink', 'like wink drink pink ink']


In [13]:
def positional_inverted_index(documents):
    index = {}
    for doc_id, doc in enumerate(documents):
        for pos, word in enumerate(doc.split()):
            if word not in index:
                index[word] = {}
            if doc_id not in index[word]:
                index[word][doc_id] = []
            index[word][doc_id].append(pos)
    return index

pos_index = positional_inverted_index(final)
print(pos_index)

{'like': {0: [0, 2], 1: [0], 2: [1], 3: [1], 4: [0]}, 'wink': {0: [1], 4: [1]}, 'drink': {0: [3], 1: [1, 2, 3], 2: [2], 3: [2], 4: [2]}, 'thing': {2: [0]}, 'ink': {2: [3], 3: [0], 4: [4]}, 'pink': {3: [3], 4: [3]}}


In [14]:
def positional_inverted_index(documents):
    index = {}
    for doc_id, doc in enumerate(documents):
        for pos, word in enumerate(doc.split()):
            if word not in index:
                index[word] = {
                    'document_frequency': 0,
                    'documents': {},
                }
            if doc_id not in index[word]['documents']:
                index[word]['documents'][doc_id] = []
                index[word]['document_frequency'] += 1
            index[word]['documents'][doc_id].append(pos)
    return index

pos_index = positional_inverted_index(final)
# print(pos_index)

In [15]:
for term, info in pos_index.items():
    print(term + ":")
    for doc_id, positions in info['documents'].items():
        print(f"\t{doc_id+1}: {', '.join(map(str, positions))}")

like:
	1: 0, 2
	2: 0
	3: 1
	4: 1
	5: 0
wink:
	1: 1
	5: 1
drink:
	1: 3
	2: 1, 2, 3
	3: 2
	4: 2
	5: 2
thing:
	3: 0
ink:
	3: 3
	4: 0
	5: 4
pink:
	4: 3
	5: 3


In [16]:
print("Term\tDocument ID\tPositions")
for term, info in pos_index.items():
    print(term + ":")
    for doc_id, positions in info['documents'].items():
        print(f"\t{doc_id+1}:\t\t{', '.join(map(str, positions))}")

Term	Document ID	Positions
like:
	1:		0, 2
	2:		0
	3:		1
	4:		1
	5:		0
wink:
	1:		1
	5:		1
drink:
	1:		3
	2:		1, 2, 3
	3:		2
	4:		2
	5:		2
thing:
	3:		0
ink:
	3:		3
	4:		0
	5:		4
pink:
	4:		3
	5:		3


In [17]:
with open('sampleindex.txt', 'w') as file:
    file.write("Term\tDocument ID\tPositions\n")
    for term, info in pos_index.items():
        file.write(term + ":\n")
        for doc_id, positions in info['documents'].items():
            file.write(f"\t{doc_id}:\t\t{', '.join(map(str, positions))}\n")

In [18]:
# pos_index = positional_inverted_index(final)

# with open('sampleindex.txt', 'w') as file:
#     file.write("Term\tDocument ID\tPositions\n")
#     for term, info in pos_index.items():
#         file.write(term + ":\n")
#         for doc_id, positions in info['documents'].items():
#             file.write(f"\t{doc_id}:\t\t{', '.join(map(str, positions))}\n")

In [19]:
pos_index = positional_inverted_index(final)

with open('sampleindex.txt', 'w') as file:
    for term, info in pos_index.items():
        file.write(term + ":\n")
        for doc_id, positions in info['documents'].items():
            file.write(f"\t{doc_id}:\t{', '.join(map(str, positions))}\n")

In [20]:
import pickle

pos_index = positional_inverted_index(final)

# Save the inverted index to a binary file
with open('sampleindex.pkl', 'wb') as file:
    pickle.dump(pos_index, file)

In [21]:
import pickle

# Load the inverted index from the binary file
with open('sampleindex.pkl', 'rb') as file:
    pos_indexs = pickle.load(file)

# print(pos_indexs)

In [22]:
# inverted index
def inverted_index(documents):
    index = {}
    for doc_id, doc in enumerate(documents):
        for word in doc.split():
            if word not in index:
                index[word] = []
            if doc_id not in index[word]:
                index[word].append(doc_id)
    return index

inv_index = inverted_index(final)
print(inv_index)

{'like': [0, 1, 2, 3, 4], 'wink': [0, 4], 'drink': [0, 1, 2, 3, 4], 'thing': [2], 'ink': [2, 3, 4], 'pink': [3, 4]}


In [23]:
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            p2 += 1
        else:
            p1 += 1
    return result

def or_postings(posting1, posting2):
    return sorted(set(posting1 + posting2))

def NOT(posting, total_docs):
    return sorted(set(range(total_docs)) - set(posting))

In [24]:
# ... (import statements and preprocessing code)

def inverted_index(documents):
    index = {}
    for doc_id, doc in enumerate(documents):
        for word in doc.split():
            if word not in index:
                index[word] = []
            if doc_id not in index[word]:
                index[word].append(doc_id)
    return index

def positional_index(documents):
    index = {}
    for doc_id, doc in enumerate(documents):
        for pos, word in enumerate(doc.split()):
            if word not in index:
                index[word] = []
            index[word].append((doc_id, pos))
    return index

inv_index = inverted_index(final)
pos_index = positional_index(final)

# ... (code for or_postings, NOT, and and_postings functions)

def process_query(query):
    query_terms = query.split()
    result = None
    for i, term in enumerate(query_terms):
        if term == "AND":
            if query_terms[i+1] == "NOT":
                result = and_postings(result, NOT(inv_index[query_terms[i+2]]))
            else:
                result = and_postings(result, inv_index[query_terms[i+1]])
        elif term == "OR":
            if query_terms[i+1] == "NOT":
                result = or_postings(result, NOT(inv_index[query_terms[i+2]]))
            else:
                result = or_postings(result, inv_index[query_terms[i+1]])
        elif term == "NOT" and i == 0:
            result = NOT(inv_index[query_terms[i+1]])
        elif i == 0:
            result = inv_index[term]
    return result

query = input('Enter your query - ')
print(process_query(query))

KeyError: 'hi'

In [25]:
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

pattern = r'TEXT: (.+)'
with open('collections/sample.txt', 'r') as file:
    txt = file.read()
    documents = re.findall(pattern, txt)

tokenization = RegexpTokenizer(r"\w+")
new = []

with open('stopWords.txt', 'r') as file:
    stopWords = file.read().split()

def preprocess(stp, stm, documents):
    for doc in documents:
        tokenized = tokenization.tokenize(doc)
        lower = [word.lower() for word in tokenized]
        if stp:
            lower = [word for word in lower if not word in stopWords]
        if stm:
            lower = [PorterStemmer().stem(word) for word in lower]
        new.append(lower)

preprocess(False, False, documents)

final = [' '.join(i) for i in new]

def inverted_index(documents):
    index = {}
    for doc_id, doc in enumerate(documents):
        for word in doc.split():
            if word not in index:
                index[word] = []
            if doc_id not in index[word]:
                index[word].append(doc_id)
    return index

inv_index = inverted_index(final)

def and_postings(posting1, posting2):
    return sorted(set(posting1).intersection(posting2))

def or_postings(posting1, posting2):
    return sorted(set(posting1).union(posting2))

def NOT(posting, total_docs):
    return sorted(set(range(total_docs)) - set(posting))

def process_query(query):
    query_terms = query.split()
    result = None
    total_docs = len(final)
    for i, term in enumerate(query_terms):
        if term == "AND":
            if query_terms[i+1] == "NOT":
                result = and_postings(result, NOT(inv_index[query_terms[i+2]], total_docs))
            else:
                result = and_postings(result, inv_index[query_terms[i+1]])
        elif term == "OR":
            if query_terms[i+1] == "NOT":
                result = or_postings(result, NOT(inv_index[query_terms[i+2]], total_docs))
            else:
                result = or_postings(result, inv_index[query_terms[i+1]])
        elif term == "NOT" and i == 0:
            result = NOT(inv_index[query_terms[i+1]], total_docs)
        elif i == 0:
            result = inv_index[term]
    return result

query = input('Enter your query - ')
print(process_query(query))

KeyError: 'hello'